In [17]:
import pandas as pd
import numpy as np
import os
from scipy.stats import kendalltau

In [4]:
submission = pd.read_csv('predict-ai-model-runtime/sample_submission.csv', sep=',')
submission

,ID,TopConfigs
0,tile:xla:d6f5f54247bd1e58a10b9e7062c636ab,0;1;2;3;4
1,tile:xla:e3a655daa38e34ec240df959b650ac16,0;1;2;3;4
2,tile:xla:f8c2c1a1098b2a361c26df668b286c87,0;1;2;3;4
3,tile:xla:4dd1716853ed46ee4e7d09ede1732de8,0;1;2;3;4
4,tile:xla:d0a69155b6340748c36724e4bfc34be3,0;1;2;3;4
...,...,...
889,layout:nlp:random:60880ed76de53f4d7a1b960b24f2...,0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...
890,layout:nlp:random:23559853d9702baaaacbb0c83fd3...,0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...
891,layout:nlp:random:f6c146fc5cf10be4f3accbaca989...,0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...
892,layout:nlp:random:32531d07a084b319dce484f53a4c...,0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...


In [23]:
for dirpath, dirnames, filenames in os.walk('predict-ai-model-runtime/npz_all/'):
    if len(filenames) == 0:
        continue
    
    print(dirpath)

predict-ai-model-runtime/npz_all/npz/layout/nlp/default/train
predict-ai-model-runtime/npz_all/npz/layout/nlp/default/test
predict-ai-model-runtime/npz_all/npz/layout/nlp/default/valid
predict-ai-model-runtime/npz_all/npz/layout/nlp/random/train
predict-ai-model-runtime/npz_all/npz/layout/nlp/random/test
predict-ai-model-runtime/npz_all/npz/layout/nlp/random/valid
predict-ai-model-runtime/npz_all/npz/layout/xla/default/train
predict-ai-model-runtime/npz_all/npz/layout/xla/default/test
predict-ai-model-runtime/npz_all/npz/layout/xla/default/valid
predict-ai-model-runtime/npz_all/npz/layout/xla/random/train
predict-ai-model-runtime/npz_all/npz/layout/xla/random/test
predict-ai-model-runtime/npz_all/npz/layout/xla/random/valid
predict-ai-model-runtime/npz_all/npz/tile/xla/train
predict-ai-model-runtime/npz_all/npz/tile/xla/test
predict-ai-model-runtime/npz_all/npz/tile/xla/valid


## Tile evaluation

In [47]:
def tile_score(candidate_configs, tile_dict):
    config_scores = tile_dict['config_runtime'] / tile_dict['config_runtime_normalizers']
    best_runtime = np.min(config_scores)
    best_candidate_runtime = np.min(config_scores[candidate_configs])
    score = 2 - best_candidate_runtime / best_runtime
    return score

In [70]:
tile_valid_dir = 'predict-ai-model-runtime/npz_all/npz/tile/xla/valid'

scores = []
tile_ids = []
for filename in os.listdir(tile_valid_dir):
    tile_id = 'tile:xla:' + filename.rstrip('.npz')
    tile_dict = dict(np.load(os.path.join(tile_valid_dir, filename)))
    n_configurations = len(tile_dict['config_runtime'])
    candidate_configs = np.random.choice(np.arange(n_configurations), min(n_configurations, 5))
    score = tile_score(candidate_configs, tile_dict)
    scores.append(score)
    tile_ids.append(tile_id)

evaluation_result = pd.DataFrame(
    data=np.stack([tile_ids, scores], axis=-1),
    columns=['ID', 'score']
)
evaluation_result

,ID,score
0,tile:xla:inception_v3_batch_128_train_e00c619c...,1.0
1,tile:xla:inception_v3_batch_128_train_73909141...,-0.7720494619485696
2,tile:xla:inception_v3_batch_128_train_3124d99c...,1.0
3,tile:xla:resnet50,0.5718114385538695
4,tile:xla:inception_v3_batch_128_train_5608c687...,0.9098023691342609
...,...,...
671,tile:xla:tf2_bert_pretrain_dynamic_batch_size_...,0.9954382205986951
672,tile:xla:resnet50,0.974838685613189
673,tile:xla:resnet_v1_50_official_batch_128_bf16_...,0.928288660314545
674,tile:xla:inception_v3_batch_128_train_31b34c3a...,0.9377060892503848


## Layout evaluation

In [32]:
def layout_score(candidate_order, layout_dict):
    runtimes = layout_dict['config_runtime']
    best_ranking = np.argsort(runtimes)
    score, _ = kendalltau(candidate_order, best_ranking)
    return score

In [34]:
layout_set = 'valid'
scores = []
layout_ids = []
for dirpath, dirnames, filenames in os.walk('predict-ai-model-runtime/npz_all/npz/layout'):
    if len(filenames) == 0:
        continue
    
    if dirpath.split('/')[-1] != layout_set:
        continue
        
    layout_id_prefix = ':'.join(dirpath.split('/')[-4:-1])
    for filename in os.listdir(dirpath):
        layout_id = layout_id_prefix+':'+filename[:-4]
        layout_dict = dict(np.load(os.path.join(dirpath, filename)))
        n_configurations = len(layout_dict['config_runtime'])
        candidate_order = np.random.permutation(np.arange(n_configurations))
        score = layout_score(candidate_order, layout_dict)
        scores.append(score)
        layout_ids.append(layout_id)
        
evaluation_result_layout = pd.DataFrame(
    data=np.stack([layout_ids, scores], axis=-1),
    columns=['ID', 'score']
)
evaluation_result_layout

,ID,score
0,layout:nlp:default:small_bert_bert_en_uncased_...,-0.0005446005133630227
1,layout:nlp:default:small_bert_bert_en_uncased_...,0.003655918669555461
2,layout:nlp:default:small_bert_bert_en_uncased_...,-0.0004635077565651036
3,layout:nlp:default:small_bert_bert_en_uncased_...,0.004708570867807966
4,layout:nlp:default:albert_en_xlarge_batch_size...,0.0012492488013708205
5,layout:nlp:default:small_bert_bert_en_uncased_...,-0.0015128769911681667
6,layout:nlp:default:small_bert_bert_en_uncased_...,0.000775727054613435
7,layout:nlp:default:bert_en_cased_L-12_H-768_A-...,0.0017725114394510494
8,layout:nlp:default:small_bert_bert_en_uncased_...,0.0025868069676023577
9,layout:nlp:default:small_bert_bert_en_uncased_...,0.002920631339734003


In [70]:
layout_valid_dir = 'predict-ai-model-runtime/npz_all/npz/tile/xla/valid'

scores = []
tile_ids = []
for filename in os.listdir(tile_valid_dir):
    tile_id = 'tile:xla:'+filename.split('.')[0]
    tile_dict = dict(np.load(os.path.join(tile_valid_dir, filename)))
    n_configurations = len(tile_dict['config_runtime'])
    candidate_configs = np.random.choice(np.arange(n_configurations), min(n_configurations, 5))
    score = tile_score(candidate_configs, tile_dict)
    scores.append(score)
    tile_ids.append(tile_id)

evaluation_result = pd.DataFrame(
    data=np.stack([tile_ids, scores], axis=-1),
    columns=['ID', 'score']
)
evaluation_result

,ID,score
0,tile:xla:inception_v3_batch_128_train_e00c619c...,1.0
1,tile:xla:inception_v3_batch_128_train_73909141...,-0.7720494619485696
2,tile:xla:inception_v3_batch_128_train_3124d99c...,1.0
3,tile:xla:resnet50,0.5718114385538695
4,tile:xla:inception_v3_batch_128_train_5608c687...,0.9098023691342609
...,...,...
671,tile:xla:tf2_bert_pretrain_dynamic_batch_size_...,0.9954382205986951
672,tile:xla:resnet50,0.974838685613189
673,tile:xla:resnet_v1_50_official_batch_128_bf16_...,0.928288660314545
674,tile:xla:inception_v3_batch_128_train_31b34c3a...,0.9377060892503848
